In [1]:
import requests
import csv

# CONFIGURE THESE VALUES:
GITLAB_BASE_URL = "https://csegitlab.engineering.unt.edu"
ACCESS_TOKEN = "FTjD2DWGZYFyxwNx6zGy"
ASSIGNMENT_KEYWORD = "graph"  # Change this to your assignment name or keyword
YOUR_USERNAME = "am3543"  # Your username on GitLab to check your role

# Headers for authentication
headers = {
    "Private-Token": ACCESS_TOKEN
}

def get_projects(page=1, per_page=100):
    url = f"{GITLAB_BASE_URL}/api/v4/projects"
    params = {
        "membership": True,         # Only projects you have access to
        "per_page": per_page,
        "page": page,
        "order_by": "last_activity_at",
        "sort": "desc",
        "archived": False
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json(), response.headers

def get_project_members(project_id):
    url = f"{GITLAB_BASE_URL}/api/v4/projects/{project_id}/members/all"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def main():
    all_projects = []
    page = 1

    print("Fetching projects...")
    while True:
        projects, headers = get_projects(page=page)
        if not projects:
            break
        all_projects.extend(projects)
        # Pagination check
        if 'X-Next-Page' in headers and headers['X-Next-Page'] == '':
            break
        page += 1

    print(f"Total projects fetched: {len(all_projects)}")

    filtered_projects = [p for p in all_projects if ASSIGNMENT_KEYWORD.lower() in p['name'].lower()]

    print(f"Projects filtered by keyword '{ASSIGNMENT_KEYWORD}': {len(filtered_projects)}")

    # Prepare output list
    output = []

    for proj in filtered_projects:
        proj_id = proj['id']
        proj_name = proj['name']
        proj_owner = proj['owner']['username'] if 'owner' in proj and proj['owner'] else "N/A"

        members = get_project_members(proj_id)

        # Check if YOUR_USERNAME is a Maintainer in that project
        your_role = None
        for m in members:
            if m['username'] == YOUR_USERNAME:
                your_role = m['access_level']  # Numeric code of access level
                # Convert numeric access_level to role name
                # See https://docs.gitlab.com/ee/api/members.html#valid-access-levels
                roles = {
                    10: "Guest",
                    20: "Reporter",
                    30: "Developer",
                    40: "Maintainer",
                    50: "Owner"
                }
                your_role = roles.get(your_role, f"Level {your_role}")
                break

        if your_role == "Maintainer" or your_role == "Owner":
            output.append({
                "Project Name": proj_name,
                "Project Owner": proj_owner,
                "Your Role": your_role
            })

    if not output:
        print("No projects found where you are a Maintainer or Owner for this assignment.")
        return

    # Save output to CSV
    csv_file = f"{ASSIGNMENT_KEYWORD}_projects_with_you_as_maintainer.csv"
    with open(csv_file, "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["Project Name", "Project Owner", "Your Role"])
        writer.writeheader()
        writer.writerows(output)

    print(f"Results saved to {csv_file}")

if __name__ == "__main__":
    main()

Fetching projects...


SSLError: HTTPSConnectionPool(host='csegitlab.engineering.unt.edu', port=443): Max retries exceeded with url: /api/v4/projects?membership=True&per_page=100&page=1&order_by=last_activity_at&sort=desc&archived=False (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)')))